# vps
> VPS provisioning: cloud-init generation, Hetzner hcloud CLI wrapper, SSH deployment helpers

In [ ]:
#| default_exp vps

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os, json, subprocess, tempfile
from pathlib import Path
from fastcloudinit.core import cloud_init_config
from dockr.core import _CLI
from dockr.compose import Compose

## Cloud-init generation

`vps_init()` builds a cloud-init YAML string for a fresh VPS. It delegates to `fastcloudinit.cloud_init_config` which handles UFW (deny incoming, allow 22/80/443), user creation, and SSH key setup. Pass `docker=True` to install Docker via `get.docker.com`, and `cf_token` to install a Cloudflare tunnel.

In [ ]:
#| export
_DOCKER_CMDS = [
    'curl -fsSL https://get.docker.com | sh',
    'usermod -aG docker {username}',
    'systemctl enable --now docker',
]

def vps_init(hostname, pub_keys, username='deploy', docker=True,
             cf_token=None, packages=None, cmds=None, **kw):
    'Cloud-init YAML for a fresh VPS: user, UFW, optional Docker + Cloudflare tunnel'
    rcmds = list(cmds or [])
    if docker:
        rcmds = [c.format(username=username) for c in _DOCKER_CMDS] + rcmds
    if cf_token:
        rcmds += [
            'curl -L -o cloudflared.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb',
            'dpkg -i cloudflared.deb',
            f'cloudflared service install {cf_token}',
        ]
    return cloud_init_config(
        hostname=hostname, username=username, pub_keys=pub_keys,
        packages=['curl'] + list(packages or []),
        cmds=rcmds, **kw)

In [ ]:
yaml = vps_init('myserver', 'ssh-rsa AAAA...', docker=True)
assert '#cloud-config' in yaml
assert 'get.docker.com' in yaml
assert 'deploy' in yaml
print('vps_init OK')
print(yaml[:400])

## Hetzner (hcloud CLI)

`callhcloud()` and `Hcloud` follow the same pattern as `calldocker()`/`Docker` and `callmultipass()`/`Multipass` — subprocess wrapper plus kwargs-to-flags dispatch.

In [ ]:
#| export
def callhcloud(*args):
    'Run hcloud CLI command, return stdout.'
    return subprocess.run(('hcloud',) + args, capture_output=True, text=True, check=True).stdout.strip()

class Hcloud(_CLI):
    'Wrap hcloud CLI: __getattr__ dispatches subcommands, kwargs become flags'
    def _run(self, cmd, *args): return callhcloud(cmd, *args)

hc = Hcloud()

In [ ]:
#| export
def hcloud_auth(token, name='default'):
    'Configure hcloud CLI with token. Call once before using create/servers/etc.'
    cfg_dir = Path.home() / '.config' / 'hcloud'
    cfg_dir.mkdir(parents=True, exist_ok=True)
    cfg = cfg_dir / 'cli.toml'

    # Read existing config (Python 3.11+ tomllib; fallback to empty on 3.10 or missing file)
    try:
        import tomllib
        data = tomllib.loads(cfg.read_text()) if cfg.exists() else {}
    except ImportError:
        data = {}

    contexts = data.get('contexts', [])
    for c in contexts:
        if c['name'] == name: c['token'] = token; break
    else: contexts.append({'name': name, 'token': token})

    with cfg.open('w') as f:
        f.write(f'active_context = "{name}"\n\n')
        for c in contexts:
            f.write('[[contexts]]\n')
            f.write(f'  name = "{c["name"]}"\n')
            f.write(f'  token = "{c["token"]}"\n\n')

In [ ]:
#| export
def create(name, image='ubuntu-24.04', server_type='cx22', location=None,
           cloud_init=None, ssh_keys=None):
    'Create a Hetzner server. cloud_init: YAML string or file path. Returns IP.'
    args = ['--name', name, '--image', image, '--type', server_type]
    if location: args += ['--location', location]
    for k in (ssh_keys or []): args += ['--ssh-key', k]
    tmp = None
    if cloud_init:
        if os.path.isfile(str(cloud_init)):
            args += ['--user-data-from-file', str(cloud_init)]
        else:
            fd, tmp = tempfile.mkstemp(suffix='.yaml')
            with os.fdopen(fd, 'w') as f: f.write(cloud_init)
            args += ['--user-data-from-file', tmp]
    try: callhcloud('server', 'create', *args)
    finally:
        if tmp: Path(tmp).unlink(missing_ok=True)
    return server_ip(name)

def servers():
    'List Hetzner servers as [{name, ip, status}]'
    data = json.loads(callhcloud('server', 'list', '-o', 'json'))
    return [{'name': s['name'], 'ip': s['public_net']['ipv4']['ip'],
             'status': s['status']} for s in data]

def server_ip(name) -> str:
    'Get public IPv4 of a Hetzner server'
    data = json.loads(callhcloud('server', 'describe', name, '-o', 'json'))
    return data['public_net']['ipv4']['ip']

def delete(name):
    'Delete a Hetzner server by name'
    callhcloud('server', 'delete', name)

## SSH helpers

Pure subprocess-based SSH/rsync utilities — no paramiko dependency. `deploy()` syncs a Compose stack to a remote host and brings it up.

In [ ]:
#| export
def _ssh_base(host, user, key, port):
    args = ['ssh', '-o', 'StrictHostKeyChecking=accept-new']
    if key: args += ['-i', str(key)]
    if port != 22: args += ['-p', str(port)]
    return args + [f'{user}@{host}']

def run_ssh(host, *cmds, user='deploy', key=None, port=22):
    'Run one or more commands on a remote host via SSH. Returns stdout.'
    return subprocess.run(_ssh_base(host, user, key, port) + [' && '.join(cmds)],
                          capture_output=True, text=True, check=True).stdout.strip()

def sync(src, dst_path, host, user='deploy', key=None, port=22):
    'Rsync local path to remote host:dst_path'
    args = ['rsync', '-az', '--delete']
    if key: args += ['-e', f'ssh -i {key}']
    subprocess.run(args + [str(src), f'{user}@{host}:{dst_path}'], check=True)

def deploy(compose, host, user='deploy', key=None, path='/srv/app', pull=False):
    'Sync Compose stack to remote host and run docker compose up -d. Returns stdout.'
    run_ssh(host, f'mkdir -p {path}', user=user, key=key)
    with tempfile.NamedTemporaryFile(suffix='.yml', mode='w', delete=False) as f:
        f.write(str(compose) if isinstance(compose, Compose) else compose)
        tmp = f.name
    try: sync(tmp, f'{path}/docker-compose.yml', host, user=user, key=key)
    finally: Path(tmp).unlink(missing_ok=True)
    if pull: run_ssh(host, f'cd {path} && docker compose pull', user=user, key=key)
    return run_ssh(host, f'cd {path} && docker compose up -d', user=user, key=key)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()